In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW
from gpt_model import GPT
from vocab import tokens
import torch.nn as nn

In [2]:
# Lista de frases como listas de tokens
all_tokens = [
    ["vocês", "podem",   "explorar", "a", "floresta", "durante", "o", "dia"],
    ["ele",   "vai",     "vender",   "um","livro",    "no",      "mercado","amanhã"],
    ["nós",   "estamos", "felizes",  "porque","tivemos","uma","experiência","interessante","na","universidade"]
]


# Inicializa as listas de inputs e targets
inputs, targets = [], []

# Para cada frase, acumula pares (contexto → próximo token)
for tokens in all_tokens:
    for i in range(1, len(tokens)):
        # contexto: tokens até a posição i (excluindo i)
        inputs.append(tokens[:i])
        print(tokens[:i])
        # target: token na posição i
        targets.append(tokens[i])
        print(tokens[i])

# Exemplo de saída:
# inputs  = [["o"], ["o","gato"], ["o","gato","dorme"], ..., ["nós","vai","para","escola"]]
# targets = ["gato","dorme","em", ..., "amanhã"]


['vocês']
podem
['vocês', 'podem']
explorar
['vocês', 'podem', 'explorar']
a
['vocês', 'podem', 'explorar', 'a']
floresta
['vocês', 'podem', 'explorar', 'a', 'floresta']
durante
['vocês', 'podem', 'explorar', 'a', 'floresta', 'durante']
o
['vocês', 'podem', 'explorar', 'a', 'floresta', 'durante', 'o']
dia
['ele']
vai
['ele', 'vai']
vender
['ele', 'vai', 'vender']
um
['ele', 'vai', 'vender', 'um']
livro
['ele', 'vai', 'vender', 'um', 'livro']
no
['ele', 'vai', 'vender', 'um', 'livro', 'no']
mercado
['ele', 'vai', 'vender', 'um', 'livro', 'no', 'mercado']
amanhã
['nós']
estamos
['nós', 'estamos']
felizes
['nós', 'estamos', 'felizes']
porque
['nós', 'estamos', 'felizes', 'porque']
tivemos
['nós', 'estamos', 'felizes', 'porque', 'tivemos']
uma
['nós', 'estamos', 'felizes', 'porque', 'tivemos', 'uma']
experiência
['nós', 'estamos', 'felizes', 'porque', 'tivemos', 'uma', 'experiência']
interessante
['nós', 'estamos', 'felizes', 'porque', 'tivemos', 'uma', 'experiência', 'interessante']
na
['

In [3]:
model = GPT(inputs[0])

In [4]:
model.predict_next_token()

'me'

In [5]:
list(zip(inputs, targets))[0]

(['vocês'], 'podem')

In [6]:
#model.tokens_vocab

In [9]:
# supondo:
# - dataloader retorna pares (tokens_batch, target_batch), onde:
#     tokens_batch: lista de tokens (ou tensor de índices) com shape (T,)
#     target_batch: índice do token seguinte, tensor escalar ou shape (1,)
# - model é uma instância de GPT
# - optimizer e criterion foram definidos (e.g. nn.CrossEntropyLoss)
num_epochs = 5
criterion = nn.CrossEntropyLoss()
loss_history = []
for epoch in range(num_epochs):
    epoch_loss = 0.0

    for input_tokens, target_token in zip(inputs, targets):
        model = GPT(input_tokens)
        # 1) configura os índices de input no modelo
        model.tokens_idx(input_tokens)

        # 2) forward para obter embedding do último token
        last_hidden = model.forward()           # tensor shape (n_embd,)

        # 3) projeção final para logits
        logits = model.predict_logits(last_hidden)  # shape (vocab_size,)

        # 4) compute loss (CrossEntropyLoss espera logits shape (C,) or (1,C) e target single int)
        target = torch.tensor(model.tokens_vocab[target_token])
        loss = criterion(torch.tensor(logits), target)

        # 5) backward + step
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

        epoch_loss += loss.item()

    # média da época e registro
    avg_loss = epoch_loss / len(inputs)
    loss_history.append(avg_loss)
    print(f"Época {epoch+1}/{num_epochs} — Loss: {avg_loss:.4f}")


C:\Users\alber\AppData\Local\Temp\ipykernel_9844\253647130.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = criterion(torch.tensor(logits), target)


Época 1/5 — Loss: 5.4712
Época 2/5 — Loss: 5.6101
Época 3/5 — Loss: 5.7878
Época 4/5 — Loss: 5.6802
Época 5/5 — Loss: 5.3877


In [ ]:
loss = nn.CrossEntropyLoss()
loss(torch.tensor([[0.1, 2.9]]), torch.tensor([1]))

In [ ]:
num_epochs = 2

In [ ]:
# 1) Converte tokens → índices
inputs_idx  = [[vocab[t] for t in seq] for seq in inputs]
targets_idx = [vocab[t]          for t in targets]


X = [pad_truncate(seq, context_length, pad_id) for seq in inputs_idx]
Y = targets_idx  # cada Y[i] já é um escalar

# 3) Tensores e DataLoader
X = torch.tensor(X, dtype=torch.long)   # (N, context_length)
Y = torch.tensor(Y, dtype=torch.long)   # (N,)
ds = TensorDataset(X, Y)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True)

# 4) Otimizador
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)

# 5) Scheduler com warmup + linear decay
def lr_lambda(current_step):
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return max(
        0.0,
        float(total_steps - current_step) / float(max(1, total_steps - warmup_steps))
    )

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


In [ ]:

# 6) Loop de treinamento
model.train()
for epoch in range(num_epochs):
    for step, (batch_x, batch_y) in enumerate(loader):
        # forward
        model.tokens_idx([ idx_to_token[id.item()] for id in batch_x[0] ])  # adapte se necessário
        logits = model.forward()        # retorna (n_embd,), se seu forward for assim ajuste
        # aqui, se seu forward retorna apenas último token, use batchization:
        # logits = model(batch_x)  # se tiver implementado batch no forward
        # assume logits: (batch_size, vocab_size) ou (vocab_size,) + reshape
        # loss
        loss = F.cross_entropy(logits.view(-1, model.vocab_size),
                               batch_y.view(-1),
                               ignore_index=pad_id)
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        #if step % 50 == 0:
        print(f"Epoch {epoch+1} | Step {step}/{len(loader)} | Loss {loss.item():.4f}")

    # opcional: validação aqui

# ao final, salve checkpoint
torch.save({
    "model": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "epoch": epoch,
    "scheduler": scheduler.state_dict(),
}, "checkpoint.pth")


# Treinamento